In [1]:
import pandas as pd
import deepchem as dc
import numpy as np
from tqdm import tqdm
import torch

/home/daffaizzuddin/miniconda3/envs/interna/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-08 03:43:37.441801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 03:43:38.026599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
import data_utils as HIVLoader

df = HIVLoader.load_main_data()
df.head()

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,False
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,False
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,False
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,False
4,O=S(=O)(O)CCS(=O)(=O)O,CI,False


In [3]:
train_ds, test_ds = HIVLoader.scaffold_split(df)
print(f'total: {len(train_ds):5d}, active: {sum(train_ds.y):4d}, inactive: {sum(train_ds.y == False):5d}')
print(f'total: {len(test_ds):5d}, active: {sum(test_ds.y):4d}, inactive: {sum(test_ds.y == False):5d}')

[03:43:45] WARNING: not removing hydrogen atom without neighbors
[03:43:45] WARNING: not removing hydrogen atom without neighbors


total: 32901, active: 1219, inactive: 31682
total:  8226, active:  211, inactive:  8015


In [4]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
transformer = dc.trans.FeaturizationTransformer(featurizer=featurizer)

train_feat = train_ds.transform(transformer)
test_feat = test_ds.transform(transformer)


[03:45:36] WARNING: not removing hydrogen atom without neighbors
[03:45:36] WARNING: not removing hydrogen atom without neighbors


In [5]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

In [6]:
model = dc.models.GCNModel(mode='classification', n_tasks=1, learning_rate=0.001, device=torch.device('cuda'))

EPOCH = 10
BATCH_SIZE = 64
metrics = [dc.metrics.Metric(metric) for metric in [roc_auc_score, f1_score, accuracy_score]]

for i in range(EPOCH):
  losses = []
  for X, y, w, _ in tqdm(train_feat.iterbatches(BATCH_SIZE), total=np.ceil(len(train_feat)/BATCH_SIZE)):
    loss = model.fit_on_batch(X, y, w)
    losses.append(loss)
  
  result_metrics = model.evaluate(test_feat, metrics, transformers=[transformer])
  result_metrics['train_loss'] = np.mean(np.array(losses))
  result_metrics['epoch'] = i
  print(result_metrics)

100%|██████████| 515/515.0 [00:20<00:00, 24.57it/s]


{'roc_auc_score': 0.7624124494061786, 'f1_score': 0.09691629955947137, 'accuracy_score': 0.975079017748602, 'train_loss': 0.1251277401645496, 'epoch': 0}


100%|██████████| 515/515.0 [00:19<00:00, 26.90it/s]


{'roc_auc_score': 0.7670731418873971, 'f1_score': 0.11155378486055777, 'accuracy_score': 0.9728908339411622, 'train_loss': 0.08551204247486063, 'epoch': 1}


100%|██████████| 515/515.0 [00:23<00:00, 22.28it/s]


...

{'roc_auc_score': 0.7436327915963254, 'f1_score': 0.009433962264150943, 'accuracy_score': 0.9744711889132021, 'train_loss': 0.07257842422028986, 'epoch': 7}


100%|██████████| 515/515.0 [00:19<00:00, 26.30it/s]


{'roc_auc_score': 0.7379939272631588, 'f1_score': 0.044444444444444446, 'accuracy_score': 0.973863360077802, 'train_loss': 0.07102328974254049, 'epoch': 8}


100%|██████████| 515/515.0 [00:19<00:00, 26.16it/s]


{'roc_auc_score': 0.7470186232567491, 'f1_score': 0.338368580060423, 'accuracy_score': 0.9733770970094822, 'train_loss': 0.06947842299807093, 'epoch': 9}


In [7]:
metrics = [dc.metrics.Metric(metric) for metric in [roc_auc_score, f1_score, accuracy_score]]
model.evaluate(test_feat, metrics, transformers=[transformer])

{'roc_auc_score': 0.7470186232567491,
 'f1_score': 0.338368580060423,
 'accuracy_score': 0.9733770970094822}